In [1]:
using Pkg
Pkg.add(PackageSpec(name="JWAS", rev="master"))
#Pkg.update()
Pkg.add("DataFrames")
Pkg.add("CSV")
using JWAS,JWAS.Datasets,DataFrames,CSV


    Updating git-repo `https://github.com/reworkhow/JWAS.jl.git`
    Updating registry at `C:\Users\fmkay\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\fmkay\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\fmkay\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\fmkay\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\fmkay\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\fmkay\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\fmkay\.julia\environments\v1.7\Manifest.toml`
    Updating git-repo `https://github.com/reworkhow/JWAS.jl.git`
    Updating registry at `C:\Users\fmkay\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\fmkay\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\fmkay\.julia\environments\v1.7\Manifest.toml`
   Resolving package

In [2]:
phenotypes = CSV.read("correlate_all_snps_haplos.csv",DataFrame,delim = ',',header=true,missingstrings=["NA"])
phenotypes = dropmissing(phenotypes, :qNurlogCort)
first(phenotypes,5)

,ID,PigID,Batch,BatchNum,Company,SowID,NurEnrich,EntryAge,cNurEntryAge
,Int64,Int64,Int64?,String?,String?,String?,Bool?,Int64?,Int64?
1,4935,3386,81,81,E,E8B260,missing,22,missing
2,4936,3387,81,81,E,W3W953,missing,22,missing
3,4937,3388,81,81,E,E8B205,missing,17,missing
4,4938,3389,81,81,E,E8B411,missing,19,missing
5,4939,3390,81,81,E,3240,missing,16,missing


,ID,PigID,Batch,BatchNum,Company,SowID,NurEnrich,EntryAge,cNurEntryAge
,Int64,Int64,Int64?,String?,String?,String?,Bool?,Int64?,Int64?
1,4935,3386,81,81,E,E8B260,missing,22,missing
2,4936,3387,81,81,E,W3W953,missing,22,missing
3,4937,3388,81,81,E,E8B205,missing,17,missing
4,4938,3389,81,81,E,E8B411,missing,19,missing
5,4939,3390,81,81,E,3240,missing,16,missing


In [3]:
Hair_G5=0.151743E-01 #Additive var.
genotypes=get_genotypes("C:/Users/fmkay/Desktop/Fazhir/Hair_project/GWAS/dummy_genotypes.txt",header=true, Hair_G5, separator = ',', method="BayesB", quality_control=false, Pi = 0.999, estimatePi = false);


The delimiterd in dummy_genotypes.txt is ','. The header (marker IDs) is provided in dummy_genotypes.txt.
Genotype informatin:
#markers: 4999; #individuals: 4095
The delimiterd in dummy_genotypes.txt is ','. The header (marker IDs) is provided in dummy_genotypes.txt.
Genotype informatin:
#markers: 4999; #individuals: 4095


In [4]:
Hair_model_equation = "qNurlogCort = Batch + EntryAge + C_groundstorage + SowID + qNurPenBatch + C_ExtractionDate + C_GrindingDate + genotypes";

In [5]:
Hair_R=0.721038E-01; #Residual var
Hair_model = build_model(Hair_model_equation, Hair_R);

In [6]:
set_covariate(Hair_model, "EntryAge");

Hair_G1=0.224905E-03;
set_random(Hair_model,"qNurPenBatch",Hair_G1);
Hair_G2=0.257092E-01;
set_random(Hair_model,"SowID",Hair_G2);
Hair_G3=0.05;
set_random(Hair_model,"C_ExtractionDate",Hair_G3);
Hair_G4=0.05;
set_random(Hair_model,"C_GrindingDate",Hair_G4);


In [7]:
out1=runMCMC(Hair_model,phenotypes, missing_phenotypes=true, estimateScale=true, chain_length=1000,burnin = 0,output_heritability=true,  output_samples_frequency=10, output_samples_for_all_parameters = false);


running MCMC ... 100%|███████████████████████████████████| Time: 0:00:22
running MCMC ... 100%|███████████████████████████████████| Time: 0:00:22


The folder results already exists.
The folder results1 is created to save results.
Checking genotypes...
Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Predicted values for individuals of interest will be obtained as the summation of Any[] (Note that genomic data is always included for now).Missing values are found in column Batch for some observations.Effects of this variable on such observations are considered as zeros. It will be used in the estimation of this effect. Users may impute missing values before the analysis 
Missing values are found in column EntryAge for some observations.Effects of this variable on such observations are considered as zeros. It will be used in the estimation of this effect. Users may impute missing values before the analysis 
Missing values are found in column SowID for some observations.Effects of this variable on such observations are considered as zeros. It will be used in the estimation of th

In [9]:
cd("C:/Users/fmkay/Desktop/Fazhir/Hair_project/GWAS/results1")
out, winVarProps = GWAS(Hair_model,"dummy_mapfile.txt","MCMC_samples_marker_effects_genotypes_qNurlogCort.txt",header=true,GWAS=true,threshold = 0.001,window_size="0.25 Mb",genetic_correlation=true,output_winVarProps = true)
CSV.write("winVarProps.csv",out)
pwd()

running GWAS... 100%|████████████████████████████████████| Time: 0:00:11
running GWAS... 100%|████████████████████████████████████| Time: 0:00:11


Compute the posterior probability of association of the genomic window that explains more than 0.001 of the total genetic variance.
Compute the posterior probability of association of the genomic window that explains more than 0.001 of the total genetic variance.


"C:\\Users\\fmkay\\Desktop\\Fazhir\\Hair_project\\GWAS\\results1"

"C:\\Users\\fmkay\\Desktop\\Fazhir\\Hair_project\\GWAS\\results1"

In [ ]:
?outputMCMCsamples

In [ ]:
?runMCMC

In [ ]:
?GWAS
